# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import folium
import urllib.parse
import hvplot.pandas as hvplot


# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,16.64,68,64,0.45,US,1699383261
1,1,keszthely,46.7681,17.2432,9.94,100,0,2.57,HU,1699383262
2,2,puerto ayora,-0.7393,-90.3518,28.98,75,6,4.02,EC,1699383262
3,3,happy valley-goose bay,53.3168,-60.3315,-1.83,59,100,3.09,CA,1699383262
4,4,college,64.8569,-147.8028,-7.03,84,100,0.00,US,1699383262


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
# Create a base map
m = folium.Map(location=[0, 0], zoom_start=2)  # You can adjust the initial center and zoom level as needed

# Iterate through the DataFrame and add markers for each city with circle size based on humidity
for index, row in city_data_df.iterrows():
    city_name = row['City']
    lat = row['Lat']
    lng = row['Lng']
    humidity = row['Humidity']
    
    # Define a color based on humidity (you can customize this as needed)
    color = 'blue' if humidity < 50 else 'green' if humidity < 70 else 'red'
    
    # Add a circle marker for each city
    folium.CircleMarker(
        location=[lat, lng],
        radius=humidity / 60,  # Adjust the scaling factor as needed
        color=color,
        fill=True,
        fill_opacity=0.6,
        popup=f"{city_name}, Humidity: {humidity}%",
    ).add_to(m)

m  # Display the map


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Define the criteria for your ideal weather conditions
max_temp_lower_bound = 21
max_temp_upper_bound = 27
max_wind_speed = 4.5
max_cloudiness = 0

# Apply the conditions to filter the DataFrame
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > max_temp_lower_bound) &
    (city_data_df['Max Temp'] < max_temp_upper_bound) &
    (city_data_df['Wind Speed'] < max_wind_speed) &
    (city_data_df['Cloudiness'] == max_cloudiness)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data from the filtered DataFrame
ideal_weather_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,siwa oasis,29.2041,25.5195,25.01,41,0,2.33,EG,1699383265
40,40,tamanrasset,22.7850,5.5228,21.95,10,0,3.09,DZ,1699383269
68,68,hamilton,39.1834,-84.5333,23.76,40,0,3.09,US,1699383210
98,98,isla vista,34.4133,-119.8610,22.94,51,0,3.60,US,1699383281
197,197,tanumah,27.1000,44.1333,25.50,30,0,2.73,SA,1699383293
203,203,jodhpur,26.2867,73.0300,25.05,28,0,2.61,IN,1699383294
275,275,tshabong,-26.0500,22.4500,22.62,15,0,4.40,BW,1699383304
299,299,al kharijah,25.4514,30.5464,26.08,17,0,4.27,EG,1699383311
317,317,kushtia,23.9011,89.1221,21.56,59,0,2.17,BD,1699383315
349,349,awjilah,29.1081,21.2869,24.18,34,0,3.26,LY,1699383323


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head(25)

,City,Country,Lat,Lng,Humidity,Hotel Name
0,albany,US,42.6001,-73.9662,68,
1,keszthely,HU,46.7681,17.2432,100,
2,puerto ayora,EC,-0.7393,-90.3518,75,
3,happy valley-goose bay,CA,53.3168,-60.3315,59,
4,college,US,64.8569,-147.8028,84,
5,iqaluit,CA,63.7506,-68.5145,79,
6,keflavik,IS,64.0049,-22.5624,70,
7,boa vista,BR,2.8197,-60.6733,52,
8,luocheng,CN,29.3848,104.0286,74,
9,edinburgh of the seven seas,SH,-37.0676,-12.3116,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [63]:
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters (10 kilometers)
categories = "accommodation.hotel"  # Category for hotels
limit = 1  # Limit results to one hotel per city
api_key = geoapify_key  # Your Geoapify API key

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Define the "bias" and "filter" parameters
    bias = f"proximity:{lng},{lat}"
    filters = f"circle:{lng},{lat},{radius}"
    
    # Create the params dictionary
    params = {
        "radius": radius,
        "categories": categories,
        "limit": limit,
        "apiKey": api_key,
        "bias": bias,
        "filter": filters
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the existing "Hotel Name" column of the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] #= name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found" in the existing "Hotel Name" column.
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Reset the index of the DataFrame
    hotel_df.reset_index(drop=True, inplace=True)   

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
albany - nearest hotel: No hotel found
keszthely - nearest hotel: City Hotel Éden
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
happy valley-goose bay - nearest hotel: Hotel North Two
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
iqaluit - nearest hotel: Frobisher Inn
keflavik - nearest hotel: Núpan Deluxe
boa vista - nearest hotel: Aipana Plaza Hotel
luocheng - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF
dawesville - nearest hotel: Baymont by Wyndham Thomasville
mattighofen - nearest hotel: Helpfauer Hof
nabire - nearest hotel: Hotel Anggrek
hsinchu - nearest hotel: 柿子紅快捷旅店
port alfred - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
saudarkrokur - nearest hotel: Mikligarður
siwa oasis - nearest hotel: فندق الكيلانى
saint-joseph - nearest hotel: PALM Hotel & Spa
olonkinbyen - nearest hotel: No hotel found
waitan

,City,Country,Lat,Lng,Humidity,Hotel Name,"(0, Hotel Name)"
0,albany,US,42.6001,-73.9662,68,No hotel found,No hotel found
1,keszthely,HU,46.7681,17.2432,100,City Hotel Éden,No hotel found
2,puerto ayora,EC,-0.7393,-90.3518,75,Hostal La Mirada De Solitario George,No hotel found
3,happy valley-goose bay,CA,53.3168,-60.3315,59,Hotel North Two,No hotel found
4,college,US,64.8569,-147.8028,84,BEST WESTERN PLUS Pioneer Park Inn,No hotel found
...,...,...,...,...,...,...,...
561,taiynsha,KZ,53.8478,69.7639,51,"Гостиница ""Орбита""",No hotel found
562,bandarbeyla,SO,9.4942,50.8122,83,No hotel found,No hotel found
563,mount isa,AU,-20.7333,139.5000,64,Ibis Styles,No hotel found
564,lazaro cardenas,MX,17.9583,-102.2000,70,Hotel Sol del Pacífico,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [57]:
# Filter the DataFrame to include only cities with hotels
cities_with_hotels_df = hotel_df[hotel_df['Hotel Name'] != 'No hotel found']

# Create a map centered around a specific location (e.g., the first city in the DataFrame)
m = folium.Map(location=[cities_with_hotels_df['Lat'].iloc[0], cities_with_hotels_df['Lng'].iloc[0]], zoom_start=3)

# Add markers for cities with hotels
for index, row in cities_with_hotels_df.iterrows():
    hotel_name = row['Hotel Name']
    city_name = row['City']
    country_name = row['Country']
    lat = row['Lat']
    lng = row['Lng']
    
    # Create a marker with hotel name, city name, and country as popup text
    marker = folium.Marker([lat, lng], popup=f"{hotel_name}<br>{city_name}, {country_name}")
    
    # Add the marker to the map
    marker.add_to(m)

m  # Display the map


In [62]:
test = hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
print(test)

Серафима
